<a href="https://colab.research.google.com/github/vikpy/DLSem3/blob/master/Assignment_8_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Download dataset of indian elephant, african elephant and tusker (100 images each). Find which pretrained model get highest accuracy**